In [1]:
%matplotlib qt5
import numpy as np
#from fcmeans import FCM
from matplotlib import pyplot as plt

# Saving KMeans Clustering Masks

In [24]:
cl_3 = np.load('kmean_clusters/clusters_3.npy')
blank_img_veg = np.zeros(cl_3.shape)
blank_img_water = np.zeros(cl_3.shape)
blank_img_sand = np.zeros(cl_3.shape)

Vegetation - [227,186,234]
water - [222,236,246]
sand - [216,202,211]

In [25]:
blank_img_veg[np.where((cl_3 == [227,186,234]).all(axis = 2))] = [1,1,1]
blank_img_water[np.where((cl_3 == [222,236,246]).all(axis = 2))] = [1,1,1]
blank_img_sand[np.where((cl_3 == [216,202,211]).all(axis = 2))] = [1,1,1]
plt.imshow(blank_img_veg)
plt.show()

In [12]:
np.save('kmean_masks/vegetation.npy',blank_img)

In [17]:
np.save('kmean_masks/water.npy',blank_img_water)
np.save('kmean_masks/sand.npy',blank_img_sand)

# Saving Fuzzy C Means Masks

In [18]:
cl_3 = np.load('fuzz_clusters/cluster_3.npy')
blank_img_veg = np.zeros(cl_3.shape)
blank_img_water = np.zeros(cl_3.shape)
blank_img_sand = np.zeros(cl_3.shape)
plt.imshow(cl_3)
plt.show()

vegetation - [24,59,26], water - [9,19,31], sand - [82,62,58]

In [21]:
blank_img_veg[np.where((cl_3 == [24,59,26]).all(axis = 2))] = [1,1,1]
blank_img_water[np.where((cl_3 == [9,19,31]).all(axis = 2))] = [1,1,1]
blank_img_sand[np.where((cl_3 == [82,62,58]).all(axis = 2))] = [1,1,1]
plt.imshow(blank_img_veg)
plt.show()

In [20]:
np.save('fuzz_masks/water.npy',blank_img_water)
np.save('fuzz_masks/sand.npy',blank_img_sand)
np.save('fuzz_masks/vegetation.npy',blank_img_veg)

# Calculating Boundary pixel density

### Actual Boundary Pixel

In [2]:
ann_2008 = np.load('D:\Kaziranga\KNP2008_actual_mask.npy')
ann_2018 = np.load('D:\Kaziranga\KNP2018_actual_mask.npy')

In [3]:
import cv2
from skimage.util import img_as_ubyte

In [5]:
edges1 = cv2.Canny(img_as_ubyte(ann_2008),150,200)
edges2 = cv2.Canny(img_as_ubyte(ann_2018),100,200)

In [7]:
edge_2008 = (edges1>0)*1
edge_2018 = (edges2>0)*1

In [11]:
np.sum(edge_2008.flatten())

2244

In [12]:
np.sum(edge_2018.flatten())

2223

In [14]:
edge_2008.shape[0]*edge_2008.shape[1]

1533694

In [15]:
2244/1533694

0.0014631341062819572

### Canny Boundary Pixel 

In [19]:
band1_array = np.load('KNP_2008_bands/band1.npy')
band6_array = np.load('KNP_2008_bands/band6.npy')
band4_array = np.load('KNP_2008_bands/band4.npy')
band2_array = np.load('KNP_2008_bands/band2.npy')
image = np.zeros((band1_array.shape[0],band1_array.shape[1],3))
image[:,:,0] = band6_array
image[:,:,1] = band4_array
image[:,:,2] = band2_array

plt.imshow(image.astype('int'))
plt.axis('off')
plt.show()

In [20]:
import cv2

In [70]:
edges1 = cv2.Canny(band4_array,150,250)
edges2 = cv2.Canny(band6_array,150,200)

In [71]:
plt.imshow(edges1, cmap = 'gray')
plt.axis('off')
plt.show()

In [72]:
edge_2008 = (edges1>0)*1
edge_2018 = (edges2>0)*1

In [73]:
np.sum(edge_2008.flatten())

19555

In [74]:
19555/1533694

0.012750261786249408

### Robert Boundary Pixel

In [90]:
kernelx = np.array([[1,0],[0,-1]])
kernely = np.array([[0,1],[-1,0]])

reg1_robertx = cv2.filter2D(band4_array,-1,kernelx)
reg1_roberty = cv2.filter2D(band4_array,-1,kernely)

reg1_robert = reg1_robertx+reg1_roberty

plt.imshow(reg1_robert>40, cmap = 'gray')
plt.axis('off')
plt.show()

In [85]:
edge_2008 = (reg1_robert>55)*1
np.sum(edge_2008.flatten())

3878

In [87]:
3878/1533694

0.0025285356792163235

### Sobel Boundary Pixel 

In [100]:
reg1_sobelx = cv2.Sobel(band4_array.astype('uint8'),cv2.CV_8U,0,1,ksize=1)
reg1_sobely = cv2.Sobel(band4_array.astype('uint8'),cv2.CV_8U,1,0,ksize=1)
reg1_sobel = reg1_sobelx+reg1_sobely

plt.imshow(reg1_sobely>20, cmap = 'gray')
plt.axis('off')
plt.show()

In [97]:
edge_2008 = (reg1_sobely>20)*1
np.sum(edge_2008.flatten())

29170

In [99]:
29170/1533694

0.0190194393405725

In [101]:
sobelx64f = cv2.Sobel(band4_array.astype('uint8'),cv2.CV_64F,1,0,ksize=1)
abs_sobel64f = np.absolute(sobelx64f)
sobel_8u = np.uint8(abs_sobel64f)



In [103]:
plt.imshow(sobel_8u>29, cmap = 'gray')
plt.axis('off')
plt.show()

### Phase Congruency Edge

In [104]:
from phasepack import phasecong,phasecongmono
from skimage.color import rgb2gray

In [105]:
reg1 = image
plt.imshow(reg1.astype('int'))
plt.axis('off')
plt.show()

In [106]:
reg1_phcg = rgb2gray(reg1)

In [107]:
plt.imshow(reg1_phcg.astype('int'))
plt.axis('off')
plt.show()

In [108]:
reg1_phcg_float = reg1_phcg.astype('float64')

(pc, ori, ft, T) = phasecongmono(reg1_phcg_float, nscale=4, minWaveLength=3, mult=2,sigmaOnf=0.55, k=3, cutOff=0.5, g=10,deviationGain=1.5, noiseMethod=-1)


In [109]:
plt.imshow(pc, cmap='gray')
plt.axis('off')
plt.show()

In [110]:
reg1_phcg_float = reg1_phcg.astype('float64')

(M,m,a,b,c,d,e) = phasecong(reg1_phcg_float)

In [115]:
plt.imshow(M, cmap='gray')
plt.axis('off')
plt.show()

In [112]:
from skimage.filters import apply_hysteresis_threshold

In [135]:
edges_phcg = apply_hysteresis_threshold(pc,0.2,0.3)

In [136]:
plt.imshow(edges_phcg, cmap='gray')
plt.axis('off')
plt.show()

In [137]:
edge_2008 = (edges_phcg)*1
np.sum(edge_2008.flatten())

95624

In [139]:
95624/1533694

0.06234881273578693